In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mail-m/MAIL.csv
/kaggle/input/mail-test/MAIL_test.csv


In [13]:
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
url = '/kaggle/input/mail-m/MAIL.csv'
df = pd.read_csv(url,header= None)
df_clean = df

In [14]:
y = df_clean[0].values
X = df_clean.drop([0],axis = 1).values

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X, y)


In [15]:
from sklearn.feature_selection import GenericUnivariateSelect, chi2
gn = GenericUnivariateSelect( mode='k_best', param=200)

In [16]:
gn.fit(X_res,y_res)

GenericUnivariateSelect(mode='k_best', param=200)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.15, random_state=1)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

In [19]:
X_train = gn.transform(X_train)
X_test = gn.transform(X_test)
X_val = gn.transform(X_val)

In [20]:
X_train.shape

(3443, 200)

In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

In [22]:
import tensorflow as tf
tf.compat.v1.reset_default_graph() # Clear Model
np.random.seed(1) # Fix Randomness
tf.compat.v1.random.set_random_seed(1) # Fix Randomness
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(202, activation='relu',input_shape=(200,)),
  tf.keras.layers.Dense(75, activation='relu'),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dropout(0.2), # คือวิธีการ dropout  0.2คือ ตัดสัญญาณ 20 %
  tf.keras.layers.Dense(1, activation='sigmoid')    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 202)               40602     
_________________________________________________________________
dense_1 (Dense)              (None, 75)                15225     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                210       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
train_acc = list()
val_acc = list()
for i in range(0,500):
  history = model.fit(X_train_norm, y_train, epochs= 1, batch_size = 50, validation_data= (X_val_norm, y_val))
  tmp_avg = np.mean(history.history['accuracy'])
  tmp_avg_val = np.mean(history.history['val_accuracy'])
  train_acc.append(tmp_avg)
  val_acc.append(tmp_avg_val)

69/69 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 0.9997 - val_loss: 1.4295 - val_accuracy: 0.8339


69/69 [==============================] - 0s 3ms/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 1.4904 - val_accuracy: 0.8503


69/69 [==============================] - 0s 3ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 2.6490 - val_accuracy: 0.8553


69/69 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 0.9991 - val_loss: 2.2719 - val_accuracy: 0.8520


69/69 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 0.9985 - val_loss: 1.7108 - val_accuracy: 0.8470


69/69 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 0.9980 - val_loss: 3.1987 - val_accuracy: 0.8553


69/69 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 0.9991 - val_loss: 3.3741 - val_accuracy: 0.8520


69/69 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 0.9988 - val_loss: 2.0240 - val_accuracy: 0.8635


In [25]:
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == y_test)/len(y_test)

0.8503496503496504

In [26]:
history.history

{'loss': [0.0027679908089339733],
 'accuracy': [0.9988382458686829],
 'val_loss': [2.023988723754883],
 'val_accuracy': [0.8634868264198303]}

# **Test**

In [30]:
url = "/kaggle/input/mail-test/MAIL_test.csv"
df_test = pd.read_csv(url,header= None)

In [31]:
ytest = df_test[0].values
Xtest = df_test.drop(([0]),axis = 1).values

In [32]:
Xtest = gn.transform(Xtest)

In [33]:
X_test_norm = scaler.transform(Xtest)

In [34]:
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == ytest)/len(ytest)

NameError: name 'clf' is not defined